# Exercício 2 - Enunciado

Considere-se de novo o algoritmo estendido de Euclides apresentado no TP2  mas usando o tipo dos inteiros e um parâmetro $N>0$
```
    INPUT  a, b : Int
    assume  a > 0 and b > 0 and a < N and b < N
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
    OUTPUT r, s, t
```

Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides

1. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função  $\gcd$  é   $\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$ .
2. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .
3. Construa codificações do programa LPA através de transformadores de predicados: “weakest pre-condition” e “strongest post-condition”. 
4. Prove a correção  do programa LPA em ambas as codificações.

# Exercício 2 - Solução

```python
assert r = gcd(a,b) and r_prime = 0 and gcd(a,b) = a * s + b * t
```

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *
from random import randrange

def prove(f):
    with Solver(name="z3") as s:
        s.add_assertion(Not(f))
        print(f.serialize())
        if s.solve():
            print(s.get_model())
            print("Failed to prove.")
        else:
            print("Proven.")
            
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

def inv(r, s, t, a, b, N, r_prime):
    return And(GE(r_prime, Int(0)), 
               Or(LE(r_prime, Int(a)), 
                  LE(r_prime, Int(b))), 
               Or(LE(r, Int(a)), 
                  LE(r, Int(b))), 
               LT(Int(0), r), 
               LT(r,Int(N)), 
               Equals(r, Plus(Times(Int(a), s), Times(Int(b), t))))

@static_vars(counter = 0)
def contraexemplo(r, s, t, a, b, N, rP):
    contraexemplo.counter+=1;
    r_prime = Symbol("r"+str(contraexemplo.counter), INT)
    s_prime = Symbol("s"+str(contraexemplo.counter), INT)
    t_prime = Symbol("t"+str(contraexemplo.counter), INT)
    #return ForAll([r_prime, s_prime, t_prime], Implies(Not(Equals(r, Int(1))), Implies(And(LT(Int(0), r_prime), LT(r_prime, r)), Not(inv(r_prime, s_prime, t_prime, a, b, N, rP)))))
    return And(LT(Int(0), r_prime), LT(r_prime, r), inv(r_prime, s_prime, t_prime, a, b, N, rP))
    #return Implies(And(LT(Int(0), r_prime), LE(r_prime, r)), Implies(inv(r_prime, s_prime, t_prime, a, b, N, rP), Equals(r, r_prime)))
    #return And(LT(Int(0), r_prime), LE(r_prime, r), Implies(inv(r_prime, s_prime, t_prime, a, b, N, rP), Equals(r, r_prime)))

N = int(input("> N: "))
a = int(randrange(1,N-1))
b = int(randrange(1,N-1))

> N: 10


$$
\mathsf{Comando}\mathbin{\;::=\;} \mathit{var}\gets \mathit{exp}\;|\;\mathsf{havoc}\,\mathit{var}\;|\;\mathsf{assert}\,\varphi\;|\;\mathsf{assume}\,\phi
$$
$$
\mathsf{Fluxo}\mathbin{\;::=\;} \mathsf{skip}\;|\;\mathsf{Comando}\,;\,\mathsf{Fluxo}\;|\;\mathsf{Fluxo}\,\|\,\mathsf{Fluxo}
$$

### Weakest pre-condition

A denotação `[C]` associa a cada fluxo `C` um predicado que caracteriza a sua correcção em termos lógicos (a sua VC) segundo a técnica WPC, sendo calculada pelas seguintes regras.

$
\begin{array}{l}
[{\sf skip}] = True \\
[{\sf assume}\:\phi] = True \\
[{\sf assert}\:\phi] = \phi \\
[ x = e ] = True \\
[(C_1 || C_2)] = [C_1] \wedge [C_2] \\
\\
[{\sf skip}\, ; C] = [C] \\
[{\sf assume}\:\phi\, ; C] = \phi \to [C] \\
[{\sf assert}\:\phi\, ; C] = \phi \wedge [C] \\
[ x = e \, ; C] = [C][e/x] \\
[(C_1 || C_2)\, ; C] = [(C_1 ; C) || (C_2 ; C)] \\
\\
[{\sf havoc}\; x \; ; C] = \forall x. \,[C]
\end{array}
$


```
    INPUT  a, b : Int
    assume  a > 0 and b > 0 and a < N and b < N
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
    OUTPUT r, s, t
```

``` python

Programa de fluxos.

Sejam pre = a > 0 and b > 0 and a < N and b < N
      inv = (r > 0) and (r < N) and (r = a*s + b*t)
      pos = r == gcd(a,b) and r_prime == 0 and gcd(a,b) == a * s + b * t
        
assume pre;
r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1;
assert inv;
havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t';
((assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False) || assume(r' == 0) and inv)
assert pos;
```

Denotação lógica com WPC.
```python
[
assume a > 0 and b > 0 and a < N and b < N;
r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1;
assert inv;
havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t';
((assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False) || assume(r' == 0) and inv)
assert r = gcd(a,b) and r_prime = 0 and gcd(a,b) = a * s + b * t
]
=
pre -> [ assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; ...; assert pos ] [a/r][b/r'][1/s][0/s'][0/t][1/t']
=
pre -> inv[a/r][b/r'][1/s][0/s'][0/t][1/t'][rP<-r][sP<-s][tP<-t] and (forall r,r',s,s',t,t'. [...; assert pos]) [a/r][b/r'][1/s][0/s'][0/t][1/t'][rP<-r][sP<-s][tP<-t]
=
pre -> inv[a/r][b/r'][1/s][0/s'][0/t][1/t'][rP<-r][sP<-s][tP<-t] and
       (forall r,r',s,s',t,t'.
           ( (r' != 0 and inv) -> inv[r/rP][s/sP][t/tP][(tP-q*t')/t'][t'/t][(sP-q*s')/s'][s'/s][(rP-q*r')/r'][r'/r][(r/r')/q]) )
             and
             ( (not(r' != 0) and inv) -> pos)
       )
```

In [2]:
r = Symbol('r', INT)
r_prime = Symbol('r_prime', INT)
s = Symbol('s', INT)
s_prime = Symbol('s_prime', INT)
t = Symbol('t', INT)
t_prime = Symbol('t_prime', INT)
q = Symbol('q', INT)

In [3]:
pre = And(GT(Int(a), Int(0)), GT(Int(b), Int(0)), GT(Int(N), Int(a)), GT(Int(N), Int(a)))

pos = And(inv(r, s, t, a, b, N, r_prime), Not(contraexemplo(r,s,t,a,b,N, r_prime)), Equals(r_prime, Int(0)))

ini = substitute(inv(r,s,t,a,b,N,r_prime), {r:Int(a), r_prime:Int(b), s:Int(1), s_prime:Int(0), t:Int(0), t_prime:Int(1)})

pres = Implies(And(Not(Equals(r_prime, Int(0))), inv(r,s,t,a,b,N,r_prime)), 
               substitute(substitute(inv(r, s, t, a, b, N,r_prime), 
                                     {r: r_prime, 
                                      r_prime: Minus(r, Times(q, r_prime)), 
                                      s: s_prime,
                                      s_prime: Minus(s, Times(q, s_prime)),
                                      t: t_prime,
                                      t_prime: Minus(t, Times(q, t_prime))}
                                    ), 
                          {q: Div(r, r_prime)}
                         )
              )
util = Implies(And(Equals(r_prime, Int(0)), inv(r, s, t, a, b, N,r_prime)), pos)
vc = Implies(pre, And(ini, ForAll([r,r_prime,s,s_prime,t,t_prime, q], And(pres, util))))
print(a, b)
prove(ini)
prove(util)
prove(pres)
prove(vc)

3 1
((0 <= 1) & ((1 <= 3) | (1 <= 1)) & ((3 <= 3) | (3 <= 1)) & (0 < 3) & (3 < 10) & (3 = ((3 * 1) + (1 * 0))))
Proven.
(((r_prime = 0) & ((0 <= r_prime) & ((r_prime <= 3) | (r_prime <= 1)) & ((r <= 3) | (r <= 1)) & (0 < r) & (r < 10) & (r = ((3 * s) + (1 * t))))) -> (((0 <= r_prime) & ((r_prime <= 3) | (r_prime <= 1)) & ((r <= 3) | (r <= 1)) & (0 < r) & (r < 10) & (r = ((3 * s) + (1 * t)))) & (! ((0 < r1) & (r1 < r) & ((0 <= r_prime) & ((r_prime <= 3) | (r_prime <= 1)) & ((r1 <= 3) | (r1 <= 1)) & (0 < r1) & (r1 < 10) & (r1 = ((3 * s1) + (1 * t1)))))) & (r_prime = 0)))
s1 := 0
t1 := 1
s := 0
t := 2
r1 := 1
r := 2
r_prime := 0
Failed to prove.
(((! (r_prime = 0)) & ((0 <= r_prime) & ((r_prime <= 3) | (r_prime <= 1)) & ((r <= 3) | (r <= 1)) & (0 < r) & (r < 10) & (r = ((3 * s) + (1 * t))))) -> ((0 <= (r - ((r / r_prime) * r_prime))) & (((r - ((r / r_prime) * r_prime)) <= 3) | ((r - ((r / r_prime) * r_prime)) <= 1)) & ((r_prime <= 3) | (r_prime <= 1)) & (0 < r_prime) & (r_prime < 10) & (r

SolverReturnedUnknownResultError: 

### Strongest post-condition

Na abordagem SPC a denotação de um fluxo com um comando de atribuição introduz um quantificador existencial, o que não é adequado à verificação com SMT solvers: 
$ \quad [ C \; ; x = e ] \; =  \; \exists a. (x = e[a/x]) \wedge [C][a/x] $

Para lidar com este problema pode-se converter o programa original ao formato "*single assignment*" (SA).
Num programa SA cada variável só pode ser usada depois de ser atribuida e só pode ser atribuída uma única vez.

Um programa (onde variáveis são atribuídas mais do que uma vez) pode ser reescrito num programa SA criando "clones" distintos das variáveis de forma a que seja possível fazer uma atribuição única a cada instância.

Neste caso, a denotação `[C]` associa a cada fluxo `C` um predicado que caracteriza a sua correcção em termos lógicos (a sua VC) segundo a técnica SPC, sendo calculada pelas seguintes regras.

$
\begin{array}{l}
[{\sf skip}] = True \\
[{\sf assume}\:\phi] = \phi \\
[{\sf assert}\:\phi] = \phi \\
[x = e ] = (x = e)\\
[(C_1 || C_2)] = [C_1] \vee [C_2] \\
\\
[C \, ; {\sf skip}\;] = [C] \\
[C \, ;{\sf assume}\:\phi] = [C] \wedge \phi \\
[C \, ;{\sf assert}\:\phi] = [C] \to \phi \\
[ C \, ; x = e ] = [C] \wedge (x = e)\\
[C\,; (C_1 || C_2)] = [(C ; C_1) || (C; C_2)] \\
\\
[S\,;\,\mathsf{havoc}\,\upsilon]\quad\;\;\equiv\quad \exists\,a\,\centerdot\,[S][\upsilon/a]
\end{array}
$



Denotação lógica com SPC
```python
[assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; ((assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False) || assume(r' == 0) and inv)
assert pos;
]
= 
[assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; ((assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False) || assume(r' == 0) and inv)] -> pos
=
[(assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; (assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False)) || (assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; assume(r' == 0) and inv)] -> pos
=
([(assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; (assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False))] or [(assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; assume(r' == 0) and inv)]) -> pos
=
([assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; (assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv;) -> False] or [(assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; assume(r' == 0) and inv)]) -> pos
```

In [ ]:
a = Symbol('a',INT)
b = Symbol('b',INT)
r = Symbol('r', INT)
r_prime = Symbol('r_prime', INT)
s = Symbol('s', INT)
s_prime = Symbol('s_prime', INT)
t = Symbol('t', INT)
t_prime = Symbol('t_prime', INT)
N = Symbol('N', INT)
q = Symbol('q', INT)
gcd = Symbol('gcd', FunctionType(INT,[INT,INT]))
aux = Symbol('aux', INT)

pre = And(GT(a, Int(0)), GT(b, Int(0)), GT(N, a), GT(N, b))

pos = And(Equals(r, gcd(a, b)), Equals(r_prime, Int(0)), Exists([s, t], Equals(gcd(a, b), Plus(Times(a, s), Times(b, t)))))

inv = And(GT(r, Int(0)), GT(N, r), Equals(r, Plus(Times(a, s), Times(b, t))))
ini = substitute(inv, {r:a, r_prime:b, s:Int(1), s_prime:Int(0), t:Int(0), t_prime:Int(1)})
pres = Implies(And(Not(Equals(r_prime, Int(0))), inv), substitute(substitute(substitute(substitute(substitute(substitute(substitute(substitute(substitute(substitute(inv, {tP:t}), {sP:s}), {rP:r}), {t_prime: Minus(tP, Times(q, t_prime))}), {t: t_prime}), {s_prime: Minus(sP, Times(q, s_prime))}), {s: s_prime}), {r_prime: Minus(rP, Times(q, r_prime))}), {r: r_prime}), {q: Div(r, r_prime)}))
util = Implies(And(Equals(r_prime, Int(0)), inv), pos)
vc = Implies(pre, And(ini, ForAll([r,r_prime,s,s_prime,t,t_prime,rP,sP,tP], And(pres, util))))

prove(Implies(axioms, vc))